In [1]:
import os
from dotenv import load_dotenv
import pandas as pd


In [2]:
load_dotenv()
csv_path = os.getenv('DATA_RAW_PATH')

In [ ]:
df = pd.read_csv(csv_path)
df

In [4]:
df['date_purchase'] = pd.to_datetime(df['date_purchase'])


In [5]:
df['time_purchase'] = pd.to_datetime(df['time_purchase'], format='%H:%M:%S', errors='coerce').dt.time


In [6]:
negativos = df[df['gmv_success'] < 0]
print(f"→ Existem {len(negativos)} valores negativos em gmv_success")
nulos = df[df['gmv_success'].isna()]
print(f"→ Existem {len(nulos)} valores nulos em gmv_success")


→ Existem 7 valores negativos em gmv_success
→ Existem 0 valores nulos em gmv_success


In [7]:
df = df[df['gmv_success'] >= 0]

In [8]:
colunas = ['place_origin_departure', 'place_origin_return', 'place_destination_departure', 'place_destination_return']
for coluna in colunas:
    # Pega só os valores únicos diferentes de zero
    rodoviarias_unicas = df.loc[df[coluna] != '0', coluna].unique()
    
    # Cria o mapa só para esses valores
    mapa = {hash_: f'Rodoviária {i+1}' for i, hash_ in enumerate(rodoviarias_unicas)}
    
    # Aplica o mapeamento apenas onde for diferente de zero
    df.loc[df[coluna] != '0', coluna] = df.loc[df[coluna] != '0', coluna].map(mapa)


C:\Users\pedro\AppData\Local\Temp\ipykernel_41056\3592278818.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df[coluna] != '0', coluna] = df.loc[df[coluna] != '0', coluna].map(mapa)
C:\Users\pedro\AppData\Local\Temp\ipykernel_41056\3592278818.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df[coluna] != '0', coluna] = df.loc[df[coluna] != '0', coluna].map(mapa)


In [9]:
clientes = df['fk_contact'].unique()
mapa_clientes = {hash_: f'Cliente {i+1}'
                    for i, hash_ in enumerate(clientes)}
mapa_clientes

df['fk_contact'] = df['fk_contact'].map(mapa_clientes)

C:\Users\pedro\AppData\Local\Temp\ipykernel_41056\960107525.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fk_contact'] = df['fk_contact'].map(mapa_clientes)


In [10]:
df.to_parquet('transacoes_preparado.parquet')